In [0]:
import keras as kr
import numpy as np
import tensorflow as tf

from keras.datasets import cifar100

from sklearn.model_selection import train_test_split

from random import randint
import time
import os
from keras.utils import to_categorical

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

#if not os.path.exists('data/models/'):
#  os.mkdir('data/models')
       

In [0]:
#if not os.path.exists('data/models/'):
#  os.mkdir('data/models')

In [0]:
# The number of coarse categories
coarse_categories = 20

# The number of fine categories
fine_categories = 100

In [0]:
#Import Dataset
(X, y_c), (x_test, y_test_c) = cifar100.load_data(label_mode='coarse')
(X, y), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [0]:
#Fine-To-Coarse Mapping
fine2coarse = np.zeros((fine_categories,coarse_categories))
for i in range(coarse_categories):
    index = np.where(y_test_c[:,0] == i)[0]
    fine_cat = np.unique([y_test[j,0] for j in index])
    for j in fine_cat:
        fine2coarse[j,i] = 1

In [0]:
# create antistoixo pinaka mono me auta pou theloume
indecies = [84, 5, 20, 25, 94, 6, 7, 14, 18, 24] 
fine2coarseData = np.zeros((10,2))
                         
for i in range(10):
  if fine2coarse[indecies[i],6] == 1.0:
    fine2coarseData[i,0] = fine2coarse[indecies[i],6]
  elif fine2coarse[indecies[i],7] == 1.0: 
    fine2coarseData[i,1] = fine2coarse[indecies[i],7]                  

# The number of coarse categories
coarse_categories = 2

# The number of fine categories
fine_categories = 10


In [0]:
#fine2coarseData

In [0]:
#APO OLO TO SET KRATAME MONO AUTA POU MAS ENDIAFEROUN

#SET gia ta fines
# TODO only fine
indices_train = np.where( (y == 84) | (y == 5) | (y == 20)|(y == 25)|(y == 94)|(y == 6)|(y == 7)|(y == 14)|(y == 18)|(y == 24))[0]
indices_test = np.where((y_test == 84) | (y_test == 5) | (y_test == 20)|(y_test == 25)|(y_test == 94)|(y_test == 6)|(y_test == 7)|(y_test == 14)|(y_test == 18)|(y_test == 24))[0]

# Reduce class labels
y_train_f = np.array(y[indices_train])
y_test_f = np.array(y_test[indices_test])
# Reduce train- and test-data
x_train = X[indices_train]
x_test2 = x_test[indices_test]
#giati pano me ta indices den to dexete
x_test = x_test2
  
#SET gia ta coarses
# Get indices to keep the right categories from dataset
indices_train = np.where( (y_c == 6) | (y_c == 7))[0]
indices_test = np.where((y_test_c == 6) | (y_test_c == 7))[0]

# Reduce class labels
y_train_c = np.array(y_c[indices_train])
y_test_c = np.array(y_test_c[indices_test])
#TA X EINAI IDIA EITE EINAI FINE EITE COARSE


#GIA TO ONE HOT ENCODING
y_train_f[np.array(y_train_f==84)]=0
y_train_f[np.array(y_train_f==5)]=1
y_train_f[np.array(y_train_f==20)]=2
y_train_f[np.array(y_train_f==25)]=3  
y_train_f[np.array(y_train_f==94)]=4
y_train_f[np.array(y_train_f==6)]=5  
y_train_f[np.array(y_train_f==7)]=6
y_train_f[np.array(y_train_f==14)]=7  
y_train_f[np.array(y_train_f==18)]=8
y_train_f[np.array(y_train_f==24)]=9 
y_test_f[np.array(y_test_f==84)]=0
y_test_f[np.array(y_test_f==5)]=1 
y_test_f[np.array(y_test_f==20)]=2
y_test_f[np.array(y_test_f==25)]=3 
y_test_f[np.array(y_test_f==94)]=4
y_test_f[np.array(y_test_f==6)]=5 
y_test_f[np.array(y_test_f==7)]=6
y_test_f[np.array(y_test_f==14)]=7 
y_test_f[np.array(y_test_f==18)]=8
y_test_f[np.array(y_test_f==24)]=9 

y_train_c[np.array(y_train_c==6)]=0
y_train_c[np.array(y_train_c==7)]=1 
y_test_c[np.array(y_test_c==6)]=0
y_test_c[np.array(y_test_c==7)]=1

#METATROPI SE ONE HOT
y_train_c = to_categorical(y_train_c, 2)
y_test_c = to_categorical(y_test_c, 2)
y_train_f = to_categorical(y_train_f, 10)
y_test_f = to_categorical(y_test_f, 10)

In [0]:
################################################################################
#    Title: ZCA
################################################################################
#    Description: 
#        This function applies ZCA Whitening to the image set
#    
#    Parameters:
#        x_1           Array of MxNxC images to compute the ZCA Whitening
#        x_2           Array of MxNxC images to apply the ZCA transform
#        num_batch    Number of batches to do the computation
# 
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def zca(x_1, x_2, epsilon=1e-5):
        
    with tf.name_scope('ZCA'):
        
        x1 = tf.placeholder(tf.float64, shape=np.shape(x_1), name='placeholder_x1')
        x2 = tf.placeholder(tf.float64, shape=np.shape(x_2), name='placeholder_x2')
        
        flatx = tf.cast(tf.reshape(x1, (-1, np.prod(x_1.shape[-3:])),name="reshape_flat"),tf.float64,name="flatx")
        sigma = tf.tensordot(tf.transpose(flatx),flatx, 1,name="sigma") / tf.cast(tf.shape(flatx)[0],tf.float64) ### N-1 or N?
        s, u, v = tf.svd(sigma,name="svd")
        pc = tf.tensordot(tf.tensordot(u,tf.diag(1. / tf.sqrt(s+epsilon)),1,name="inner_dot"),tf.transpose(u),1, name="pc")
        
        net1 = tf.tensordot(flatx, pc,1,name="whiten1")
        net1 = tf.reshape(net1,np.shape(x_1), name="output1")
        
        flatx2 = tf.cast(tf.reshape(x2, (-1, np.prod(x_2.shape[-3:])),name="reshape_flat2"),tf.float64,name="flatx2")
        net2 = tf.tensordot(flatx2, pc,1,name="whiten2")
        net2 = tf.reshape(net2,np.shape(x_2), name="output2")
        
    with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            x_1,x_2 = sess.run([net1,net2], feed_dict={x1: x_1, x2: x_2})    
    return x_1,x_2

In [129]:
#treksimo tou ZCA
time1 = time.time()
x_train,x_test = zca(x_train,x_test)
time2 = time.time()
print('Time Elapsed - ZCA Whitening: '+str(time2-time1));

Time Elapsed - ZCA Whitening: 329.8146834373474


In [0]:
#Split Training set into Training and Validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_f, test_size=.2, random_state=2019, stratify = y_train_f )
X = 0
y = 0


In [0]:
#y_train_f.shape

In [0]:
################################################################################
#    Title: Preprocess Img
################################################################################
#    Description: 
#        This function pads images by 4 pixels, randomly crops them, then
#        randomly flips them
#    
#    Parameters:
#        x_1           Array of MxNxC images to compute the ZCA Whitening
#        x_2           Array of MxNxC images to apply the ZCA transform
#        num_batch    Number of batches to do the computation
# 
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def preprocess_img(X,y):
        
    with tf.name_scope('Preproc'):
        
        images = tf.placeholder(tf.float64, shape=np.shape(X))
        labels = tf.placeholder(tf.float64, shape=np.shape(y))
        
        net = tf.map_fn(lambda img: tf.image.flip_left_right(img), images)
        net = tf.map_fn(lambda img: tf.image.rot90(img), net)
        net = tf.image.resize_image_with_crop_or_pad(net,40,40)
        net = tf.map_fn(lambda img: tf.random_crop(img, [32,32,3]), net)

        net1 = tf.image.resize_image_with_crop_or_pad(images,40,40)
        net1 = tf.map_fn(lambda img: tf.random_crop(img, [32,32,3]), net1)
        
        net = tf.concat([net, net1],0)
        net = tf.random_shuffle(net, seed=0)
        net_labels = tf.concat([labels, labels],0)
        net_labels = tf.random_shuffle(net_labels,seed=0)
        
        net = tf.map_fn(lambda img: tf.image.random_flip_up_down(img), net)
        
    with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            x_t,y_t = sess.run([net,net_labels], feed_dict={images: X, labels: y})    
    return x_t,y_t

In [133]:
#treksimo tis proepeksergasias ton eikonon
#Flip, pad and randomly crop each photo

time1 = time.time()
x_train,y_train = preprocess_img(x_train,y_train)
time2 = time.time()
print('Time Elapsed - Img Preprocessing: '+str(time2-time1));


Time Elapsed - Img Preprocessing: 12.674590587615967


In [0]:
#compile model
model = Model(inputs=in_layer,outputs=net)
sgd_coarse = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(optimizer= sgd_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#Single Classifier Training
# Constructing 1st CNN (shared layers)


from keras import optimizers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from keras.models import Model

in_layer = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

net = Conv2D(32, 3, strides=1, padding='same', activation='elu')(in_layer)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(net)
net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(net)
net = Dropout(.2)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(net)
net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(net)
net = Dropout(.3)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)


net = Flatten()(net)
net = Dense(512, activation='elu')(net)
net = Dense(10, activation='softmax')(net)

In [0]:
#tbCallBack = kr.callbacks.TensorBoard(log_dir='./data/graph/elu_drop/', histogram_freq=0, write_graph=True, write_images=True)

In [0]:
#y_train.shape

In [138]:
#train model
batch = 64
index= 0
step = 2
stop = 10

#vgike to callbacks=[tbCallBack]
while index < stop:
    model.fit(x_train, y_train, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val) )
    index += step
    model.save_weights('model_coarse'+str(index))
save_index = index

Train on 8000 samples, validate on 1000 samples
Epoch 1/2
8000/8000 [==============================] - 3s 436us/step - loss: 2.2029 - acc: 0.1809 - val_loss: 2.0256 - val_acc: 0.2490
Epoch 2/2
8000/8000 [==============================] - 2s 268us/step - loss: 1.8819 - acc: 0.2996 - val_loss: 1.6900 - val_acc: 0.3650
Train on 8000 samples, validate on 1000 samples
Epoch 3/4
8000/8000 [==============================] - 2s 267us/step - loss: 1.7049 - acc: 0.3687 - val_loss: 1.7070 - val_acc: 0.3770
Epoch 4/4
8000/8000 [==============================] - 2s 268us/step - loss: 1.5636 - acc: 0.4204 - val_loss: 1.5938 - val_acc: 0.4340
Train on 8000 samples, validate on 1000 samples
Epoch 5/6
8000/8000 [==============================] - 2s 265us/step - loss: 1.4799 - acc: 0.4561 - val_loss: 1.4428 - val_acc: 0.4700
Epoch 6/6
8000/8000 [==============================] - 2s 266us/step - loss: 1.4061 - acc: 0.4834 - val_loss: 1.4571 - val_acc: 0.4900
Train on 8000 samples, validate on 1000 sample

In [0]:
#create fine optimizer
sgd_fine = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)


In [0]:
#epidi exoume shared layers, gia na min ta ksanakpaideoume
for i in range(len(model.layers)):
    model.layers[i].trainable=False

In [0]:
#einai apo tin train test split ta y_train kai y_val
#ousiastika pairnei ta y coarses mesa apo ton pinaka geitniasis me esoteriko ginomeno
y_train_c = np.dot(y_train,fine2coarseData)
y_val_c = np.dot(y_val,fine2coarseData)

In [0]:
#print ekteleso keli pou ipologizei to y_rain_c apo pano
#print(y_train_c.shape)
#print(y_train_c)

In [0]:
#print ekteleso keli pou ipologizei to y_rain_c apo pano
#print(y_val.shape)
#print(y_val)

In [0]:
#afou ekteleso to pano keli me to dot
#y_train.shape

In [0]:
#meta to treksimo me to dot
#print(y_train_c.shape)
#print(y_train_c)

In [0]:
#Fine-Tuning for Coarse Classifier
#orismos tou 2ou sineliktikou diktiou

net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(model.layers[-8].output)
net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(512, activation='elu')(net)
out_coarse = Dense(2, activation='softmax')(net)

model_c = Model(inputs=in_layer,outputs=out_coarse)
model_c.compile(optimizer= sgd_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())

In [147]:
#train tou deuterou sinelktikou
index = 30
step = 10
stop = 40

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c))
    index += step


Train on 8000 samples, validate on 1000 samples
Epoch 31/40
8000/8000 [==============================] - 2s 305us/step - loss: 0.2680 - acc: 0.8888 - val_loss: 0.2031 - val_acc: 0.9190
Epoch 32/40
8000/8000 [==============================] - 1s 168us/step - loss: 0.2190 - acc: 0.9136 - val_loss: 0.1965 - val_acc: 0.9230
Epoch 33/40
8000/8000 [==============================] - 1s 169us/step - loss: 0.2059 - acc: 0.9166 - val_loss: 0.1935 - val_acc: 0.9240
Epoch 34/40
8000/8000 [==============================] - 1s 169us/step - loss: 0.1947 - acc: 0.9237 - val_loss: 0.1900 - val_acc: 0.9220
Epoch 35/40
8000/8000 [==============================] - 1s 167us/step - loss: 0.1852 - acc: 0.9266 - val_loss: 0.1944 - val_acc: 0.9270
Epoch 36/40
8000/8000 [==============================] - 1s 169us/step - loss: 0.1793 - acc: 0.9283 - val_loss: 0.1938 - val_acc: 0.9250
Epoch 37/40
8000/8000 [==============================] - 1s 166us/step - loss: 0.1759 - acc: 0.9336 - val_loss: 0.1985 - val_acc: 

In [148]:
model_c.compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
stop = 50

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c))
    index += step

Train on 8000 samples, validate on 1000 samples
Epoch 41/50
8000/8000 [==============================] - 3s 315us/step - loss: 0.1508 - acc: 0.9417 - val_loss: 0.2008 - val_acc: 0.9240
Epoch 42/50
8000/8000 [==============================] - 1s 168us/step - loss: 0.1522 - acc: 0.9401 - val_loss: 0.2001 - val_acc: 0.9230
Epoch 43/50
8000/8000 [==============================] - 1s 166us/step - loss: 0.1517 - acc: 0.9387 - val_loss: 0.2015 - val_acc: 0.9290
Epoch 44/50
8000/8000 [==============================] - 1s 167us/step - loss: 0.1442 - acc: 0.9449 - val_loss: 0.2003 - val_acc: 0.9260
Epoch 45/50
8000/8000 [==============================] - 1s 168us/step - loss: 0.1461 - acc: 0.9407 - val_loss: 0.2069 - val_acc: 0.9260
Epoch 46/50
8000/8000 [==============================] - 1s 168us/step - loss: 0.1449 - acc: 0.9447 - val_loss: 0.2010 - val_acc: 0.9250
Epoch 47/50
8000/8000 [==============================] - 1s 169us/step - loss: 0.1408 - acc: 0.9433 - val_loss: 0.2067 - val_acc: 

In [0]:
#Fine-Tuning for Fine Classifiers
#dimourgoume tous 2 fine classifiers gia ta dedomena mas
def fine_model():
    net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(model.layers[-8].output)
    net = Conv2D(64, 3, strides=1, padding='same', activation='elu')(net)
    net = Dropout(.6)(net)
    net = MaxPooling2D((2, 2), padding='same')(net)

    net = Flatten()(net)
    net = Dense(512, activation='elu')(net)
    out_fine = Dense(10, activation='softmax')(net)
    model_fine = Model(inputs=in_layer,outputs=out_fine)
    model_fine.compile(optimizer= sgd_coarse,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

In [0]:
fine_models = {'models' : [{} for i in range(coarse_categories)], 'yhf' : [{} for i in range(coarse_categories)]}
for i in range(coarse_categories):
    model_i = fine_model()
    fine_models['models'][i] = model_i

In [0]:
def get_error(y,yh):
#    # Threshold 
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error



In [152]:
#Train Fine Classifiers on Respective Data
for i in range(coarse_categories):
    index= 0
    step = 5
    stop = 5
    
    # Get all training data for the coarse category
    ix = np.where([(y_train[:,j]==1) for j in [k for k, e in enumerate(fine2coarseData[:,i]) if e != 0]])[1]
    x_tix = x_train[ix]
    y_tix = y_train[ix]
    
    # Get all validation data for the coarse category
    ix_v = np.where([(y_val[:,j]==1) for j in [k for k, e in enumerate(fine2coarseData[:,i]) if e != 0]])[1]
    x_vix = x_val[ix_v]
    y_vix = y_val[ix_v]
    
    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
    
    fine_models['models'][i].compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
    stop = 10

    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
        
    yh_f = fine_models['models'][i].predict(x_val[ix_v], batch_size=batch)
    print('Fine Classifier '+str(i)+' Error: '+str(get_error(y_val[ix_v],yh_f)))


Train on 4000 samples, validate on 500 samples
Epoch 1/5
4000/4000 [==============================] - 2s 518us/step - loss: 1.4422 - acc: 0.4263 - val_loss: 1.2897 - val_acc: 0.5080
Epoch 2/5
4000/4000 [==============================] - 1s 189us/step - loss: 1.1939 - acc: 0.5162 - val_loss: 1.1637 - val_acc: 0.5560
Epoch 3/5
4000/4000 [==============================] - 1s 190us/step - loss: 1.1298 - acc: 0.5420 - val_loss: 1.1576 - val_acc: 0.5760
Epoch 4/5
4000/4000 [==============================] - 1s 189us/step - loss: 1.0738 - acc: 0.5745 - val_loss: 1.1681 - val_acc: 0.5460
Epoch 5/5
4000/4000 [==============================] - 1s 189us/step - loss: 1.0445 - acc: 0.5857 - val_loss: 1.1229 - val_acc: 0.5780
Train on 4000 samples, validate on 500 samples
Epoch 6/10
4000/4000 [==============================] - 2s 496us/step - loss: 0.9896 - acc: 0.6135 - val_loss: 1.1074 - val_acc: 0.5840
Epoch 7/10
4000/4000 [==============================] - 1s 184us/step - loss: 0.9794 - acc: 0.6

In [153]:
#ipologismos accuracy gia ta cnn mas
from sklearn.metrics import confusion_matrix, accuracy_score

#ipologismos Acuuracy gia coarse genikos
y_pred = model_c.predict(x_test,batch_size=64,verbose=1)
Y_pred = np.argmax(y_pred,axis=1)
Y_test = np.argmax(y_test_c,axis=1)
accuracy = (len(Y_test) - np.count_nonzero(Y_pred - Y_test))/len(Y_test)
print('Model coarse')
print(accuracy)

#accuracy gia kathe klassi xorista
cifar_classes = ["household furniture","insects"]

cm =confusion_matrix(Y_test,Y_pred)
#print(cm)
acc_per_label = cm.diagonal()/cm.sum(axis=1)
acc_per_label_final = [cifar_classes[i]+ " = "+str(acc_per_label[i]) for i in range(0,2)]
print('Pososta ana klasi')
print(acc_per_label_final)

1000/1000 [==============================] - 1s 581us/step
Model coarse
0.9
Pososta ana klasi
['household furniture = 0.834', 'insects = 0.966']


In [154]:
#ipologismos Accuracy gia ton fine classifier gia ta epipla
#arxika kanoume probabilistic averaging me ta coarses

#apo ta y_test_f kratao mono auta gia ta epipla
Y_test = np.argmax(y_test_f,axis=1)
indices_test = np.where((Y_test == 0) | (Y_test == 1) | (Y_test == 2)|(Y_test == 3)|(Y_test == 4))[0]
x_test3 = x_test[indices_test]
y_test_f3 =  np.array(y_test_f[indices_test])

#probabilistic averaging
#iplogismos "varon" apo coarses
yh = np.zeros(np.shape(y_test_f3))
yh_c = model_c.predict(x_test3,batch_size=64,verbose=1)
print("Evaluating Fine Classifier: ", str(0))
fine_models['yhf'][0] = fine_models['models'][0].predict(x_test3, batch_size=64)
yh = np.multiply(yh_c[:,0].reshape((len(y_test_f3)),1), fine_models['yhf'][0])

#ara to yh einai to kainourgio mas y_pred, opote
Y_pred = np.argmax(yh,axis=1)
Y_test = np.argmax(y_test_f3,axis=1)
accuracy = (len(Y_test) - np.count_nonzero(Y_pred - Y_test))/len(Y_test)
#score = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('Model fine: household furniture')
print(accuracy)
#print(score)

#ipologismos gia kathe classi xorista
cifar_classes = ["table", "bed","chair", "couch","wardrobe"]

cm =confusion_matrix(Y_test,Y_pred)
#print(cm)
acc_per_label = cm.diagonal()/cm.sum(axis=1)
acc_per_label_final = [cifar_classes[i]+ " = "+str(acc_per_label[i]) for i in range(0,5)]
print('accuracies for each class seperately')
print(acc_per_label_final)


500/500 [==============================] - 0s 93us/step
Evaluating Fine Classifier:  0
Model fine: household furniture
0.396
accuracies for each class seperately
['table = 0.17', 'bed = 0.19', 'chair = 0.22', 'couch = 0.83', 'wardrobe = 0.57']


In [155]:
#ipologismos Accuracy gia ton fine classifier gia ta entoma
#gia tous fine classifiers ginetai Probabilistic Averaging me vasi ta coarses


#apo ta y_test_f kratao mono auta gia ta epipla
Y_test = np.argmax(y_test_f,axis=1)
indices_test = np.where((Y_test == 5) | (Y_test == 6) | (Y_test == 7)|(Y_test == 8)|(Y_test == 9))[0]
x_test4 = x_test[indices_test]
y_test_f4 =  np.array(y_test_f[indices_test])

#probabilistic averaging
#iplogismos "varon" apo coarses
yh = np.zeros(np.shape(y_test_f4))
yh_c = model_c.predict(x_test4,batch_size=64,verbose=1)
print("Evaluating Fine Classifier: ", str(0))
fine_models['yhf'][1] = fine_models['models'][1].predict(x_test4, batch_size=64)
yh = np.multiply(yh_c[:,0].reshape((len(y_test_f4)),1), fine_models['yhf'][1])

#ara to yh einai to kainourgio mas y_pred, opote
Y_pred = np.argmax(yh,axis=1)
Y_test = np.argmax(y_test_f4,axis=1)
accuracy = (len(Y_test) - np.count_nonzero(Y_pred - Y_test))/len(Y_test)
#score = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('Model fine: insects')
print(accuracy)
#print(score)

cifar_classes = ["bee","beetle", "butterfly","caterpillar", "cockroach"]

cm =confusion_matrix(Y_test,Y_pred)
#print(cm)
acc_per_label = cm.diagonal()/cm.sum(axis=1)
acc_per_label_final = [cifar_classes[i]+ " = "+str(acc_per_label[i]) for i in range(0,5)]
print('accuracies for each class seperately')
print(acc_per_label_final)

500/500 [==============================] - 0s 94us/step
Evaluating Fine Classifier:  0
Model fine: insects
0.568
accuracies for each class seperately
['bee = 0.22', 'beetle = 0.57', 'butterfly = 0.59', 'caterpillar = 0.72', 'cockroach = 0.74']
